In [1]:
%reset -f

In [2]:
import os
import pandas as pd
import chess.pgn
import chess.engine
import io
import asyncio
import yaml
from datetime import datetime
import platform

def analyze_chess_game(uuid: str, pgn: str, engine_path: str) -> pd.DataFrame:
    # Load the PGN
    game = chess.pgn.read_game(io.StringIO(pgn))

    if game is None:
        print(f"Warning: Failed to parse PGN for game {uuid}")
        return pd.DataFrame(columns=["uuid", "move_number", "move", "score_white"])

    # Initialize lists to hold data
    move_numbers = []
    moves = []
    scores_white = []

    # Analyze the game
    with chess.engine.SimpleEngine.popen_uci(engine_path) as engine:
        board = game.board()

        for i, move in enumerate(game.mainline_moves(), 1):
            board.push(move)
            info = engine.analyse(board, chess.engine.Limit(time=0.1))
            score_white = info["score"].white().score(mate_score=1000)

            # Append data to lists
            move_numbers.append(i)
            moves.append(move.uci())
            scores_white.append(score_white)

    # Create a DataFrame
    df = pd.DataFrame({
        "uuid": [uuid] * len(move_numbers),
        "move_number": move_numbers,
        "move": moves,
        "score_white": scores_white
    })

    return df

def analyze_multiple_games(games: pd.DataFrame, engine_path: str) -> pd.DataFrame:
    game_dfs = []
    processed_games = 0

    # Iterate over each game in the dataframe
    for _, row in games.iterrows():
        uuid = row['uuid']
        pgn = row['pgn']

        # Analyze the game and append the result to the list
        game_df = analyze_chess_game(uuid, pgn, engine_path)
        game_dfs.append(game_df)

        # Increment and print the number of processed games
        processed_games += 1
        print(f"Processed {processed_games} games", flush=True)

    # Concatenate all dataframes into one
    return pd.concat(game_dfs, ignore_index=True)

def get_stockfish_path():
    if platform.system() == "Windows":
        if hasattr(asyncio, 'WindowsProactorEventLoopPolicy'):
            asyncio.set_event_loop_policy(asyncio.WindowsProactorEventLoopPolicy())
        return "C:/Program Files/ChessEngines/stockfish_16/stockfish-windows-x86-64-avx2.exe"
    return "/usr/games/stockfish"

In [3]:
import pandas as pd
from sqlalchemy import create_engine, inspect, text

# Configuration — adjust to your local PostgreSQL setup
DB_NAME         = "sandbox"
DB_USER         = "postgres"
DB_PASSWORD     = "AsidDe5845edDikkDee"
DB_HOST         = "localhost"
DB_PORT         = "5432"

engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

# Define your table names (no backticks needed in PostgreSQL)
schema_games        = "stg_chess_com"
table_games         = "players_games"
schema_games_moves  = "stg_stockfish"
table_games_moves   = "players_games_moves"

# Helper: check if table with prefix exists
def table_with_prefix_exists(engine, schema_name, table_prefix):
    inspector = inspect(engine)
    tables = inspector.get_table_names(schema=schema_name)
    return any(t.startswith(table_prefix) for t in tables)

# Choose query based on table existence
if table_with_prefix_exists(engine, schema_games_moves, table_games_moves):
    query = f"""
    SELECT DISTINCT
        uuid,
        end_time,
        pgn,
    FROM {schema_games}.{table_games} game
    LEFT JOIN (
        SELECT DISTINCT game_uuid FROM {schema_games_moves}.{table_games_moves}
    ) games_moves
    ON game.game_uuid = games_moves.game_uuid
    WHERE 
        games_moves.game_uuid IS NULL
        AND LENGTH(game.pgn) > 0
        AND game.rules = 'chess'
    """
else:
    query = f"SELECT * FROM {schema_games}.{table_games} LIMIT 10"

# Run query and load into DataFrame
games = pd.read_sql(query, engine)
print("Query to fetch games executed successfully!")

Query to fetch games executed successfully!


In [4]:
# Calculate all games moves for all games
engine_path = get_stockfish_path()
games_moves = analyze_multiple_games(games, engine_path)

Processed 1 games
Processed 2 games
Processed 3 games
Processed 4 games
Processed 5 games
Processed 6 games
Processed 7 games
Processed 8 games
Processed 9 games
Processed 10 games


In [5]:
with engine.begin() as conn:
    conn.execute(text(f"CREATE SCHEMA IF NOT EXISTS {schema_games_moves}"))

games_moves.to_sql(
    name        = table_games_moves,
    con         = engine,
    schema      = schema_games_moves,     
    if_exists   = 'append', # If the table exists
    index       = False # Ignore the df index   
)

574